In [1]:
import pandas as pd

path_to_parquet ="C:/Users/volte/Downloads/vw_cpt_brussels_params_completeset_20250318_remapped.parquet"
df = pd.read_parquet(path_to_parquet)
df.dropna(inplace=True)



In [2]:
sonderings = list(set(df["sondering_id"]))
pick_one = df[df["sondering_id"] == sonderings[0]]
pick_one.to_csv("input/input.csv")
df.columns

Index(['sondering_id', 'index', 'pkey_sondering', 'sondeernummer', 'x', 'y',
       'start_sondering_mtaw', 'diepte_sondering_tot', 'diepte', 'diepte_mtaw',
       'qc', 'fs', 'qtn', 'rf', 'fr', 'icn', 'sbt', 'ksbt', 'lithostrat_id'],
      dtype='object')

In [3]:
input = pd.read_csv("input/input.csv")

In [4]:
x=list(set(input["x"]))[0]
y=list(set(input["y"]))[0]

df["dist"] = (df["x"] - x)**2 + (df["y"] - y)**2
minvalue = df["dist"].min()

minvalues= df[df["dist"] == minvalue]
sondids = list(set(minvalues["sondering_id"]))
closest = minvalues[minvalues["sondering_id"] == sondids[0]]
closest.to_csv("input/closest.csv")

In [1]:
import pandas as pd
import altair as alt
import paths_cpt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
# todo: if you change model type, the model needs to be imported again for pickle.load to work correctly
from sklearn.ensemble import RandomForestClassifier
import data_module_copy as data
import pickle
from scipy.stats import gaussian_kde
from scipy.ndimage import gaussian_filter1d
import matplotlib.pyplot as plt
import geopandas as gpd


segments_oi = [
"Quartair",
"Diest",
"Bolderberg",
"Sint_Huibrechts_Hern",
"Ursel",
"Asse",
"Wemmel",
"Lede",
"Brussel",
"Merelbeke",
"Kwatrecht",
"Mont_Panisel",
"Aalbeke",
"Mons_en_Pevele",
]



def smooth_one_samp(pred_labels, depths):

    #ids = validation['sondering_id'].values
    all_classes = np.unique(pred_labels)
    class_to_kde = {cls: i for i, cls in enumerate(all_classes)}
    #cmap = plt.get_cmap("tab10", len(all_classes))

   # mask = ids == id
    d = depths
    l = pred_labels

    sort_idx = np.argsort(d)
    d_sorted = d
    l_sorted = l

    classes = np.unique(l_sorted)
    depth_grid = np.linspace(d_sorted.min(), d_sorted.max(), len(d_sorted))
    density_matrix = np.zeros((len(classes), len(depth_grid)))

    for i, cls in enumerate(classes):
        cls_depths = d_sorted[l_sorted == cls]
        if len(cls_depths) > 1:
            kde = gaussian_kde(cls_depths)
            class_to_kde[cls] = kde
            density_matrix[i] = kde(depth_grid)
        else:
            density_matrix[i, np.abs(depth_grid - cls_depths[0]).argmin()] = 1.0

    max_idx = np.argmax(density_matrix, axis=0)
    smoothed_local = classes[max_idx]

    smoothed_labels = smoothed_local[np.argsort(sort_idx)]

    return smoothed_labels, class_to_kde






theoretical_ordering = {"Quartair":1,
"Diest":2,
"Bolderberg":3,
"Sint_Huibrechts_Hern":4,
"Ursel":5,
"Asse":6,
"Wemmel":7,
"Lede":8,
"Brussel":9,
"Merelbeke":10,
"Kwatrecht":11,
"Mont_Panisel":12,
"Aalbeke":13,
"Mons_en_Pevele":14}

def correct_labels(labels, seq=None, min_length=20):
    clean_run = False
    if len(labels) == 0:
        return [], [], []

    corrected = []
    observed = [labels[0]]
    lengths = []
    current_label = labels[0]
    run_length = 1

    for i in range(1, len(labels)):
            if labels[i] == current_label:
                run_length += 1
            else:
                corrected.append([current_label, run_length])
                lengths.append(run_length)
                current_label = labels[i]
                observed.append(current_label)
                run_length = 1

    corrected.append([current_label, run_length])
    lengths.append(run_length)
    
    while not clean_run:
        problems = [False, False, False, False]


        # merging too short segments
        i = 1
        while i < len(corrected):
            label, length = corrected[i]
            if length < min_length:
                corrected[i-1][1] += length
                corrected.pop(i)
                i -= 1
                problems[0] = True
            i += 1
        i = 1
        while i < len(corrected)-1:
            #if theoretical_ordering[observed[i-1]]< theoretical_ordering[observed[i]]:
                prev, p_len = corrected[i-1]
                curr, c_len = corrected[i]
                next, n_len = corrected[i+1]
                if prev == next:
                    corrected[i-1][1] = p_len + c_len + n_len
                    corrected.pop(i+1)
                    corrected.pop(i)
                    problems[1] = True
                i += 1
        i = 1
        while i < len(corrected):
            #check if adjacent are equal
                prev, p_len = corrected[i-1]
                curr, c_len = corrected[i]
                if prev == curr:
                    corrected[i-1][1] = p_len + c_len
                    corrected.pop(i)
                    problems[2]=True
                i += 1
        i = 1
        while i < len(corrected)-2:
            #if theoretical_ordering[observed[i-1]]< theoretical_ordering[observed[i]]:
                prev, p_len = corrected[i-1]
                curr, c_len = corrected[i]
                next, n_len = corrected[i+1]
                next_2, n_len_2 = corrected[i+2]
                if prev == next_2:
                    corrected[i-1][1] = p_len + c_len + n_len + n_len_2
                    corrected.pop(i+2)
                    corrected.pop(i+1)
                    corrected.pop(i)
                    problems[3] = True
                i += 1


        if not any(problems):
             clean_run = True

    not_clean = True
    while not_clean:
        second_round_prob = [False]
        i=1
        while i < len(corrected):
            
            prev, p_len = corrected[i-1]
            curr, c_len = corrected[i]
            if theoretical_ordering[curr] < theoretical_ordering[prev]:
                #corrected[i-1][0] = curr
                print("order correction occurred for : ",corrected[i])
                print("full seq is : ", corrected)
                corrected[i][0] = prev
                second_round_prob[0] = True
            i += 1

        if not any(second_round_prob):
             not_clean = False

    expanded = []
    for label, length in corrected:
        expanded.extend([label] * length)                     

    return expanded




input = data.DataSet("input/input.csv",segments_oi, 0.05).get_preprocessed(split=False, features =  ['qc', 'fs', 'qtn', 'rf', 'fr', 'icn', 'sbt', 'ksbt'])

      Unnamed: 0  sondering_id   index  \
0         244356          4614  312080   
1         244357          4614  312081   
2         244358          4614  312082   
3         244359          4614  312083   
4         244360          4614  312084   
...          ...           ...     ...   
1885      246241          4614  313965   
1886      246242          4614  313966   
1887      246243          4614  313967   
1888      246244          4614  313968   
1889      246245          4614  313969   

                                         pkey_sondering       sondeernummer  \
0     https://www.dov.vlaanderen.be/data/sondering/2...  2017-11245-S3B-003   
1     https://www.dov.vlaanderen.be/data/sondering/2...  2017-11245-S3B-003   
2     https://www.dov.vlaanderen.be/data/sondering/2...  2017-11245-S3B-003   
3     https://www.dov.vlaanderen.be/data/sondering/2...  2017-11245-S3B-003   
4     https://www.dov.vlaanderen.be/data/sondering/2...  2017-11245-S3B-003   
...                  

In [9]:
input.columns

Index(['Unnamed: 0', 'sondering_id', 'index', 'pkey_sondering',
       'sondeernummer', 'x', 'y', 'start_sondering_mtaw',
       'diepte_sondering_tot', 'diepte', 'diepte_mtaw', 'qc', 'fs', 'qtn',
       'rf', 'fr', 'icn', 'sbt', 'ksbt', 'lithostrat_id'],
      dtype='object')